In [60]:
#|default_exp losses

#|export
from abc import ABC, abstractmethod
import torch
import numpy as np
import pandas as pd
from tsai.basics import *

# Custom losses

In [61]:
#| export
class Loss(nn.Module, ABC):
    def __init__(self, reduction:str=None):
        super().__init__()
        self.reduction = reduction
    
    def _reduce(self, loss: torch.Tensor) -> torch.Tensor:
        if self.reduction == 'mean': return loss.mean()
        if self.reduction == 'sum': return loss.sum()
        return loss
    
    @abstractmethod
    def _compute_loss(self, input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        return NotImplementedError
    
    def forward(self, input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        loss = self._compute_loss(input, target)
        return self._reduce(loss)

In [62]:
#| export

class MSELoss(Loss):
    def __init__(self, reduction:str=None):
        super().__init__(reduction)

    def _compute_loss(self, input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        return (target-input)**2

class MAELoss(Loss):
    def __init__(self, reduction:str=None):
        super().__init__(reduction)

    def _compute_loss(self, input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        return torch.abs(target-input)
    
class MSLELoss(Loss):
    def __init__(self, reduction:str=None):
        super().__init__(reduction)

    def _compute_loss(self, input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        return (torch.log1p(input) - torch.log1p(target))**2
    
class HubberLoss(Loss):
    def __init__(self, reduction:str=None, delta:float=1.):
        super().__init__(reduction)
        self.delta = delta

    def _compute_loss(self, input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        error = target - input
        
        is_small_error = error < self.delta
        small_error_loss = (0.5 * (error ** 2))
        large_error_loss = (self.delta * (torch.abs(error) - 0.5 * self.delta))

        return torch.where(is_small_error, small_error_loss, large_error_loss)

## Weighted losses

In [63]:
#|export
class WeightedLoss(nn.Module, ABC):
    def __init__(self, thresholds:dict, weights:dict):
        super().__init__()

        # Activity levels' weights can be equal across all variables or different,
        # and this should be taken into account during preprocessing. 
        self.all_variables_have_same_weights = len(weights.keys()) == 1
        ranges, weights = self._preprocess_data(thresholds, weights)

        self.register_buffer('ranges', torch.Tensor(ranges))
        self.register_buffer('weights', torch.Tensor(weights))

    def weighted_loss_tensor(self, target: torch.Tensor) -> torch.Tensor:        
        batch, variables, horizon = target.shape  # Example shape (32, 4, 6)
        variable, max_range, interval = self.ranges.shape  # Example shape (4, 4, 2)

        target_shaped = torch.reshape(target, (batch, variables, 1, horizon))  # Example shape (32, 4, 6) -> (32, 4, 1, 6)
        ranges_shaped = torch.reshape(self.ranges, (variable, max_range, 1, interval))  # Example shape (4, 4, 2) -> (4, 4, 1, 2)

        weights_tensor = ((ranges_shaped[..., 0] <= target_shaped) & (target_shaped <= ranges_shaped[..., 1])).float()
             
        if self.all_variables_have_same_weights:
            equation = 'r,bvrh->bvh'
        else:
            equation = 'vr,bvrh->bvh'

        return torch.einsum(equation, self.weights, weights_tensor)
    
    @abstractmethod
    def loss_measure(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return NotImplementedError
    
    def _preprocess_data(self, thresholds, weights):
        # If each variable has its own weights, calculate the maximum size of weights.
        # Padding shorter weights with NaNs prevents heterogeneous tensor errors.
        if (self.all_variables_have_same_weights):
            ranges = np.array(list(thresholds.values())[:])
            weights = np.array(next(iter(weights.values())))
        else:
            def add_padding(x, padding_value, shape):
                result = np.full(shape, padding_value)
                for i, r in enumerate(x):
                    result[i, :len(r)] = r
                return result
            
            max_size = max([len(array) for array in thresholds.values()])

            ranges_raw = thresholds.values()
            ranges = add_padding(ranges_raw, np.nan, (len(ranges_raw), max_size, 2))

            weights_raw = [weights[key] for key in thresholds.keys()]
            weights = add_padding(weights_raw, 0.0, (len(weights_raw), max_size))

        return ranges, weights
    
    def forward(self, y_pred, y_true, reduction='mean'):
        error = self.loss_measure(y_pred, y_true)
        weights = self.weighted_loss_tensor(y_true)

        if reduction == 'mean':
            loss = (error * weights).mean()
        elif reduction == 'sum':
            loss = (error * weights).sum()
        else: 
            loss = error*weights
        
        return loss

In [64]:
# Test
device = 'cpu'
ranges = {'A': np.array([[0, 1], [1, 2], [2, 3], [3, 4]]),
          'B': np.array([[0, 1], [1, 2], [2, 3], [3, 4]]),
          'C': np.array([[0, 1], [1, 2], [2, 3], [3, 4]]),
          'D': np.array([[0, 1], [1, 2], [2, 3], [3, 4]])}

weights = {'A': np.array([1, 2, 3, 4])}

target = torch.tensor([[[0.5, 1.5, 2.5, 3.5, 4.5, 5.5],
                        [0.5, 1.5, 2.5, 3.5, 4.5, 5.5],
                        [0.5, 1.5, 2.5, 3.5, 4.5, 5.5],
                        [0.5, 1.5, 2.5, 3.5, 4.5, 5.5]]], device=device, dtype=torch.float32)

input = target + 1

expected_weights = torch.tensor([[[1, 2, 3, 4, 0, 0],
                                 [1, 2, 3, 4, 0, 0],
                                 [1, 2, 3, 4, 0, 0],
                                 [1, 2, 3, 4, 0, 0]]], device=device, dtype=torch.float32)

solact_levels = ['low', 'moderate', 'elevated', 'high']

class DummyLoss(WeightedLoss):
        def loss_measure(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
            pass

def test_LossWeightsTensor():
    loss = DummyLoss(ranges, weights).to(device)
    result = loss.weighted_loss_tensor(target)

    assert torch.equal(result, expected_weights), f"Expected {expected_weights}, but got {result}"
    print(f"Loss Tensor test passed!")

In [65]:
# Test

thresholds_ne = {
    'var1': [[0, 1], [1, 2], [2, 3]],
    'var2': [[4, 5], [5, 6]],
}

weights_ne = {
    'var1': [1, 2, 3],
    'var2': [3, 4],
}

target_ne = torch.tensor([[[0.5,0.5,0.5,1.5],
                         [4.5,4.5,5.5,4.5]]])

expected_weights_ne = torch.tensor([[[1,1,1,2],
                                   [3,3, 4, 3]]])

def test_LossWeightsTensor_different_weights():
    model = DummyLoss(thresholds_ne, weights_ne)
    loss_tensor = model.weighted_loss_tensor(target_ne)
    assert torch.equal(loss_tensor, expected_weights_ne), f"Expected {expected_weights}, but got {loss_tensor}"
    print("Test for different weights per variable passed!")

In [66]:
#|export

class wMSELoss(WeightedLoss):
    def __init__(self, thresholds, weights):
        super().__init__(thresholds, weights)

    
    def loss_measure(self, input, target):
        return MSELoss()(input, target)

In [67]:
#|export

class wMAELoss(WeightedLoss):
    def __init__(self, thresholds, weights):
        super().__init__(thresholds, weights)

    def loss_measure(self, input, target):
        return MAELoss()(input, target)

In [68]:
#|export

class wMSLELoss(WeightedLoss):
    def __init__(self, thresholds, weights):
        super().__init__(thresholds, weights)
    
    def loss_measure(self, input, target):
        return MSLELoss()(input, target)

In [69]:
#|export

class wHubberLoss(WeightedLoss):
    def __init__(self, thresholds, weights, delta=2.0):
        super().__init__(thresholds, weights)
        self.delta = delta
    
    def loss_measure(self, y_pred, y_true):
        return HubberLoss(self.delta)(y_pred, y_true)

In [70]:
#|export

class ClassificationLoss(WeightedLoss):
    def __init__(self, thresholds, loss):
        n_variables = len(thresholds.keys())
        weights = {'All': np.arange(n_variables)}

        super().__init__(ranges, weights)

        self.loss = loss
    
    def loss_measure(self, input, target):
        return self.loss(input, target)

    def forward(self, input, target, reduction='mean'):
        error = self.loss_measure(input, target)
        weights = 1 + torch.abs(self.weighted_loss_tensor(target) - self.weighted_loss_tensor(input))

        if (error.shape != weights.shape): # To avoid the use of other loss functions as CrossEntropyLoss
            weights = weights.mean(dim=1)
            
        if reduction == 'mean':
            loss = (error * weights).mean()
        elif reduction == 'sum':
            loss = (error * weights).sum()
        
        return loss

In [71]:
#| export

class TrendedLoss(nn.Module):
    def __init__(self, loss: Loss):
        super().__init__()
        self.loss = loss

    @staticmethod
    def _slope(y):
        x = np.arange(len(y))
        slope, _ = np.polyfit(x, y, deg=1)
        return slope

    @staticmethod
    def _calculate_trends(tensor):
        np_tensor = tensor.cpu().detach().numpy()
        trends = np.apply_along_axis(TrendedLoss._slope, 2, np_tensor)
        return torch.Tensor(trends)

    def forward(self, input, target):
        batch, variables, _ = input.shape

        input_trend = TrendedLoss._calculate_trends(input)
        target_trend = TrendedLoss._calculate_trends(target)
        
        trend_diff = 1 + torch.abs(input_trend - target_trend)

        error = self.loss(input, target)
        weights = trend_diff.reshape(batch,variables,1)
        loss = (error * weights).mean()

        return loss

In [72]:
# Test

def check_loss_function(loss_class, expected_value, loss_func=None):
    if loss_class.__name__ == "ClassificationLoss":
        loss = loss_class(ranges, loss_func).to(device)
    elif loss_class.__name__ == "TrendedLoss":
        loss = loss_class(loss_func).to(device)
    else:
        loss = loss_class(ranges, weights).to(device)
    
    result = loss(input, target)

    assert torch.isclose(result, expected_value), f"Expected {expected_value}, but got {result}"
    print(f"{type(loss).__name__} test passed!")

In [73]:
# Test

def test_wMSELoss():
    expected_mse_loss = torch.mean(expected_weights * (target - input) ** 2)
    check_loss_function(wMSELoss, expected_mse_loss)

def test_wMAELoss():
    expected_mae_loss = torch.mean(expected_weights * torch.abs(target - input))
    check_loss_function(wMAELoss, expected_mae_loss)

def test_wMSLELoss():
    expected_msle_loss = torch.mean(expected_weights * ((torch.log1p(target) - torch.log1p(input)) ** 2))
    check_loss_function(wMSLELoss, expected_msle_loss)

def test_wHuberLoss():
    delta = 1
    expected_hubber_loss = torch.mean(expected_weights * 
                                   torch.where(torch.abs(input - target) < delta, 
                                                0.5 * (input - target) ** 2,
                                                delta * (torch.abs(input - target) - 0.5 * delta)
                                                )
                                  )
    check_loss_function(wHubberLoss, expected_hubber_loss)



def test_ClassificationLoss():
    expected_classification_loss = MSELoss('mean')(input, target)
    check_loss_function(ClassificationLoss, expected_classification_loss, loss_func=MSELoss())

def test_TrendedLoss():
    expected_loss = torch.mean((target - input) ** 2) # The trend will be the same so the weights will be all 1
    check_loss_function(TrendedLoss, expected_loss, loss_func=MSELoss())

In [74]:
#|export

class LossMetrics:
    def __init__(self, loss_func, solact_levels):
        self.loss_func = loss_func
        self.solact_levels = solact_levels

    # Weighted Regressive Loss Metrics
    def _apply_weighted_loss_by_level(self, input, target, weight_idx):
        loss_copy = deepcopy(self.loss_func)
        
        for idx in range(len(loss_copy.weights)):
            if idx != weight_idx:
                loss_copy.weights[idx] = 0
        
        return loss_copy(input, target)
    
    
    # Classification Loss Metrics
    def _compute_misclassifications(self, predictions, targets):
        classifier = self.loss_func.weighted_loss_tensor
        true_labels = classifier(targets)
        predicted_labels = classifier(predictions)

        misclassified_labels = (true_labels != predicted_labels).int() * predicted_labels

        return misclassified_labels.unique(return_counts=True)

    def _count_misclassifications_by_level(self, predictions, targets, level):
        unique_labels, label_counts = self._compute_misclassifications(predictions, targets)
        label_count_dict = dict(zip(unique_labels.tolist(), label_counts.tolist()))

        return label_count_dict.get(level, 0)
    

    # Metrics functions
    def loss_low(self, input, target):
        return self._apply_weighted_loss_by_level(input, target, 0)
    
    def loss_moderate(self, input, target):
        return self._apply_weighted_loss_by_level(input, target, 1)
    
    def loss_elevated(self, input, target):
        return self._apply_weighted_loss_by_level(input, target, 2)
    
    def loss_high(self, input, target):
        return self._apply_weighted_loss_by_level(input, target, 3)
    
    def missclassifications_low(self, predictions, targets):
        return self._count_misclassifications_by_level(predictions, targets, 1)
    
    def missclassifications_moderate(self, predictions, targets):
        return self._count_misclassifications_by_level(predictions, targets, 2)
    
    def missclassifications_elevated(self, predictions, targets):
        return self._count_misclassifications_by_level(predictions, targets, 3)
    
    def missclassifications_high(self, predictions, targets):
        return self._count_misclassifications_by_level(predictions, targets, 4)
    

    # Metrics retrieval
    def get_metrics(self):
        if not isinstance(self.solact_levels, list):
            def Metrics_Not_Available(input, target): return '_' 
            return [Metrics_Not_Available]
        elif isinstance(self.loss_func, ClassificationLoss):
            return [self.missclassifications_low, self.missclassifications_moderate, self.missclassifications_elevated, self.missclassifications_high]
        
        elif isinstance(self.loss_func, WeightedLoss):
            return [self.loss_low, self.loss_moderate, self.loss_elevated, self.loss_high]
        
        else:
            return []


In [75]:
# Test
def test_LossMetrics():
    loss = wMAELoss(ranges, weights).to(device)
    metrics = LossMetrics(loss, solact_levels).get_metrics()

    loss_value = loss(input, target)
    metrics_values = [metric(input, target) for metric in metrics]

    assert torch.isclose(loss_value, sum(metrics_values)), f"Expected {loss_value}, but got {sum(metrics_values)} ({metrics_values})"
    print("LossMetrics test passed!")

def test_LossMetrics_for_classification():
    loss = ClassificationLoss(ranges, MSELoss()).to(device)
    metrics = LossMetrics(loss, solact_levels).get_metrics()

    total_counts = 0
    for i in range(1, 5):
        total_counts += LossMetrics(loss, solact_levels)._count_misclassifications_by_level(input, target, i) 

    metrics_values = [metric(input, target) for metric in metrics]

    assert np.isclose(total_counts, sum(metrics_values)), f"Expected {total_counts}, but got {sum(metrics_values)} ({metrics_values})"
    print("LossMetrics for classification loss test passed!")

In [76]:
#| Test
test_LossWeightsTensor()
test_LossWeightsTensor_different_weights()
test_wMSELoss()
test_wMAELoss()
test_wMSLELoss()
test_wHuberLoss()
test_LossMetrics()
test_LossMetrics_for_classification()

input -= 1 # To make it equal to target and has no effect on the loss
test_ClassificationLoss()
test_TrendedLoss()


Loss Tensor test passed!
Test for different weights per variable passed!
wMSELoss test passed!
wMAELoss test passed!
wMSLELoss test passed!
wHubberLoss test passed!
LossMetrics test passed!
LossMetrics for classification loss test passed!
ClassificationLoss test passed!
TrendedLoss test passed!


In [77]:
#|eval: false
#|hide
from nbdev import *
nbdev_export()